In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import random
import csv
import json 
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from scipy import spatial

In [2]:
torch.backends.cudnn.deterministic = True
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)
np.random.seed(123)
torch.cuda.manual_seed_all(123)

In [3]:
df = pd.read_csv('QueryType.csv')

f = open('querytype_dataset_sentence_embeddings.json')  
input_embeddings = json.load(f)

f = open('querytype_keys_sentence_embeddings.json')  
output_embeddings = json.load(f)

querytypes = set(df['Query Type'])

In [20]:
df_train = df.loc[0:1]
df_test = df.loc[0:1]
for key in querytypes:
    temp = df[df['Query Type']==key]
    temp = temp.reset_index(drop=True)
    df_train=df_train.append(temp.loc[:int(0.8*len(temp))])
    df_test=df_test.append(temp.loc[int(0.8*len(temp)):])
    
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [24]:
def get_data(df):
    x = []
    y = []
    for i in range(len(df)):
        temp = np.array(input_embeddings[str(df['ID'][i])])
        x.append(temp)
        temp = np.array(output_embeddings[df['Query Type'][i]])
        y.append(temp)

    x = np.array(x, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    
    return x, y

x_train, y_train = get_data(df_train)
x_test, y_test = get_data(df_test)

In [38]:
def training(x, y):
    model = LinearRegression().fit(x, y)
    return model

def predict(model, x):
    predicted = model.predict(x)
    return predicted

def cosine_simil(y1, y2):
    return 1-spatial.distance.cosine(y1, y2)

In [32]:
LR = training(x_train,y_train)

In [33]:
y_pred = predict(LR, x_test)

In [45]:
for i in tqdm(range(len(y_pred))):
    max_cos=0
    max_key = None
    for key in output_embeddings.keys():
        if cosine_simil(y_pred[i],output_embeddings[key]) > max_cos:
            max_cos = cosine_simil(y_pred[i],output_embeddings[key])
            max_key = key
    df_test['Predicted Query Type'][i]=max_key

  0%|                                                                                         | 0/9709 [00:00<?, ?it/s]D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|█████████████████████████████████████████████████████████████████████████████| 9709/9709 [00:47<00:00, 202.52it/s]


In [49]:
len(df_test[df_test['Query Type']==df_test['Predicted Query Type']])/len(df_test)

0.3646101555258008